In [1]:
# Enable autoreload to automatically reload modules when they are updated
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from typing import Dict, Any, Optional, Union
import joblib
import psutil
import os
import numpy as np

# Only add path if not already added
cwd = Path.cwd()
sarpyx_path = cwd.parent
if str(sarpyx_path) not in __import__('sys').path:
    __import__('sys').path.append(str(sarpyx_path))

# Import after adding path to avoid import errors
try:
    from sarpyx.processor.core.focus import CoarseRDA
    if 'CoarseRDA' not in globals() or 'raw_data' not in globals():
        print('✅ Successfully imported CoarseRDA')
except ImportError as e:
    print(f'❌ Import error: {e}')
    raise
    
def get_processed_data_paths(base_dir: Path) -> Dict[str, Path]:
    """Get the paths for echo, metadata, and ephemeris files in the processed_data directory.

    Args:
        base_dir (Path): The base directory containing the 'processed_data' folder.

    Returns:
        Dict[str, Path]: Dictionary with keys 'echo', 'metadata', and 'ephemeris' and their corresponding file paths.

    Raises:
        AssertionError: If any of the required files are not found.
    """
    processed_data_dir = base_dir / 'processed_data'
    assert processed_data_dir.exists(), f'Processed data directory not found: {processed_data_dir}'

    echo_path = next(processed_data_dir.glob('*_echo.pkl'), None)
    metadata_path = next(processed_data_dir.glob('*_metadata.pkl'), None)
    ephemeris_path = next(processed_data_dir.glob('*_ephemeris.pkl'), None)

    assert echo_path is not None, 'Echo file not found in processed_data directory.'
    assert metadata_path is not None, 'Metadata file not found in processed_data directory.'
    assert ephemeris_path is not None, 'Ephemeris file not found in processed_data directory.'

    return {
        'echo': echo_path,
        'metadata': metadata_path,
        'ephemeris': ephemeris_path,
    }

# Only load data if not already loaded
if 'raw_data' not in globals():
    print('📁 Loading processed data files...')
    paths = get_processed_data_paths(cwd.parent)
    echo_path = paths['echo']
    metadata_path = paths['metadata']
    ephemeris_path = paths['ephemeris']

    echo = joblib.load(echo_path)
    metadata = joblib.load(metadata_path)
    ephemeris = joblib.load(ephemeris_path)

    raw_data = {
        'echo': echo,
        'metadata': metadata,
        'ephemeris': ephemeris,
    }

    print('✅ Loaded data successfully:')
    print(f'  Echo shape: {echo.shape}')
    print(f'  Metadata shape: {metadata.shape}')
    print(f'  Ephemeris shape: {ephemeris.shape}')
else:
    print('✅ Data already loaded and available in workspace')

✅ Successfully imported CoarseRDA
📁 Loading processed data files...
✅ Loaded data successfully:
  Echo shape: (56130, 25724)
  Metadata shape: (56130, 53)
  Ephemeris shape: (881, 59)


In [3]:
def print_memory_usage() -> None:
    """Print the current memory usage of the Python process.

    Returns:
        None

    Raises:
        AssertionError: If memory usage cannot be retrieved.
    """
    process = psutil.Process(os.getpid())
    mem_bytes = process.memory_info().rss
    assert mem_bytes > 0, 'Memory usage could not be determined.'
    mem_mb = mem_bytes / (1024 ** 2)
    print(f'Current memory usage: {mem_mb:.2f} MB')

print_memory_usage()

Current memory usage: 11420.19 MB


In [ ]:
# Initialize the focuser
try:
    focuser = CoarseRDA(raw_data=raw_data,
                        verbose=True,
                        backend='numpy',
                    )
    print('✅ Focuser initialized successfully!')
    print(f'  Radar data shape: {focuser.radar_data.shape}')
    print(f'  Backend: {focuser._backend}')
except Exception as e:
    print(f'❌ Error initializing focuser: {e}')
    raise

Loaded radar data with shape: (56130, 25724)
Azimuth lines: 56130, Range lines: 25724
Generating transmission replica...
Range decimation code: 1
Range sample frequency: 100092592.64 Hz
TX pulse start frequency: -43801345.00 Hz
TX ramp rate: 1927378686407.00 Hz/s
TX pulse length: 0.000045 s
Number of TX values: 4549
Phase parameters - phi1: -460.87, phi2: 9.64e+11
Replica length: 4549
Transmission replica generated successfully!
Elapsed time for _generate_tx_replica: 0.0505 seconds
✅ Focuser initialized successfully!
  Radar data shape: (56130, 25724)
  Backend: numpy


: 

# SAR Focusing API

This section provides a minimal API to focus the SAR product using the CoarseRDA processor.
The API includes functions for:
- Focusing SAR products
- Displaying results
- Memory management

In [ ]:
def focus_sar_product(
    focuser: CoarseRDA, 
    save_path: Optional[Union[str, Path]] = None
) -> np.ndarray:
    """Focus SAR product using Range Doppler Algorithm.
    
    Args:
        focuser (CoarseRDA): Initialized CoarseRDA processor instance.
        save_path (Optional[Union[str, Path]]): Optional path to save the focused data.
        
    Returns:
        np.ndarray: Focused radar data array.
        
    Raises:
        AssertionError: If focuser is not properly initialized.
        Exception: If focusing process fails.
    """
    assert isinstance(focuser, CoarseRDA), 'focuser must be a CoarseRDA instance'
    assert hasattr(focuser, 'radar_data'), 'Focuser must be initialized with radar data'
    assert focuser.radar_data is not None, 'Radar data cannot be None'
    
    print('🚀 Starting SAR product focusing...')
    print(f'Input data shape: {focuser.radar_data.shape}')
    print(f'Backend: {focuser._backend}')
    
    try:
        # Run the focusing algorithm
        focuser.data_focus()
        
        # Save if path provided
        if save_path:
            save_path = Path(save_path) if isinstance(save_path, str) else save_path
            focuser.save_file(save_path)
            print(f'✅ Focused data saved to: {save_path}')
        
        print('✅ SAR focusing completed successfully!')
        return focuser.radar_data
        
    except Exception as e:
        print(f'❌ Error during focusing: {str(e)}')
        print(f'📊 Debug info:')
        print(f'  Radar data shape: {focuser.radar_data.shape}')
        if hasattr(focuser, 'effective_velocities'):
            print(f'  Effective velocities shape: {focuser.effective_velocities.shape}')
        if hasattr(focuser, 'az_freq_vals'):
            print(f'  Azimuth freq vals shape: {focuser.az_freq_vals.shape}')
        if hasattr(focuser, 'D'):
            print(f'  D array shape: {focuser.D.shape}')
        if hasattr(focuser, 'slant_range_vec'):
            print(f'  Slant range vec shape: {focuser.slant_range_vec.shape}')
        raise


def get_focusing_metadata(focuser: CoarseRDA) -> Dict[str, Any]:
    """Extract metadata information from the CoarseRDA processor.
    
    Args:
        focuser (CoarseRDA): Initialized CoarseRDA processor instance.
        
    Returns:
        Dict[str, Any]: Dictionary containing processing metadata.
    """
    metadata = {
        'backend': focuser._backend,
        'verbose': focuser._verbose,
        'data_shape': focuser.radar_data.shape,
        'range_sample_freq': getattr(focuser, 'range_sample_freq', None),
        'replica_length': getattr(focuser, 'replica_len', None),
        'azimuth_lines': focuser.len_az_line,
        'range_lines': focuser.len_range_line,
    }
    
    # Add optional attributes if they exist
    optional_attrs = ['wavelength', 'pri', 'c', 'az_sample_freq']
    for attr in optional_attrs:
        if hasattr(focuser, attr):
            metadata[attr] = getattr(focuser, attr)
    
    return metadata


# Run the focusing process
print('📁 Setting up output path...')
focused_data_path = cwd.parent / 'processed_data' / 'focused_sar_data.pkl'
focused_data_path.parent.mkdir(exist_ok=True)  # Ensure directory exists

print('📊 Processor metadata before focusing:')
pre_focus_metadata = get_focusing_metadata(focuser)
for key, value in pre_focus_metadata.items():
    print(f'  {key}: {value}')

focused_radar_data = focus_sar_product(focuser, focused_data_path)

📁 Setting up output path...
📊 Processor metadata before focusing:
  backend: numpy
  verbose: True
  data_shape: (56130, 25724)
  range_sample_freq: 100092592.64
  replica_length: 4549
  azimuth_lines: 56130
  range_lines: 25724
🚀 Starting SAR product focusing...
Input data shape: (56130, 25724)
Backend: numpy
Starting SAR data focusing...
Initial radar data shape: (56130, 25724)
Processing with w_pad=4549, original_w=25724
FFT input data shape: (56130, 25724)


In [ ]:
def display_focusing_results(radar_data: np.ndarray, metadata: Optional[Dict[str, Any]] = None) -> None:
    """Display comprehensive information about the focused radar data.
    
    Args:
        radar_data (np.ndarray): Focused radar data array.
        metadata (Optional[Dict[str, Any]]): Optional metadata dictionary.
    """
    print('=' * 50)
    print('📈 SAR FOCUSING RESULTS')
    print('=' * 50)
    
    # Basic data information
    print('📊 Data Information:')
    print(f'  Shape: {radar_data.shape}')
    print(f'  Data type: {radar_data.dtype}')
    print(f'  Size: {radar_data.nbytes / (1024**2):.2f} MB')
    print(f'  Complex data: {np.iscomplexobj(radar_data)}')
    
    # Statistical information
    print('\n📈 Statistical Analysis:')
    if np.iscomplexobj(radar_data):
        amplitudes = np.abs(radar_data)
        phases = np.angle(radar_data)
        
        print(f'  Max amplitude: {amplitudes.max():.6f}')
        print(f'  Mean amplitude: {amplitudes.mean():.6f}')
        print(f'  Min amplitude: {amplitudes.min():.6f}')
        print(f'  Std amplitude: {amplitudes.std():.6f}')
        print(f'  Phase range: [{phases.min():.3f}, {phases.max():.3f}] rad')
    else:
        print(f'  Max value: {radar_data.max():.6f}')
        print(f'  Mean value: {radar_data.mean():.6f}')
        print(f'  Min value: {radar_data.min():.6f}')
        print(f'  Std value: {radar_data.std():.6f}')
    
    # Data quality checks
    print('\n🔍 Data Quality Checks:')
    nan_count = np.isnan(radar_data).sum()
    inf_count = np.isinf(radar_data).sum()
    zero_count = (radar_data == 0).sum()
    
    if nan_count > 0:
        print(f'  ⚠️  Warning: {nan_count} NaN values detected ({nan_count/radar_data.size*100:.2f}%)')
    else:
        print('  ✅ No NaN values detected')
        
    if inf_count > 0:
        print(f'  ⚠️  Warning: {inf_count} infinite values detected ({inf_count/radar_data.size*100:.2f}%)')
    else:
        print('  ✅ No infinite values detected')
        
    if zero_count > radar_data.size * 0.1:  # More than 10% zeros might be suspicious
        print(f'  ⚠️  Note: {zero_count} zero values detected ({zero_count/radar_data.size*100:.2f}%)')
    else:
        print(f'  ✅ Zero values: {zero_count} ({zero_count/radar_data.size*100:.2f}%)')
    
    # Processing metadata
    if metadata:
        print('\n⚙️  Processing Metadata:')
        for key, value in metadata.items():
            print(f'  {key}: {value}')
    
    print('=' * 50)


def analyze_focusing_quality(radar_data: np.ndarray) -> Dict[str, float]:
    """Analyze the quality of the focused SAR data.
    
    Args:
        radar_data (np.ndarray): Focused radar data array.
        
    Returns:
        Dict[str, float]: Dictionary containing quality metrics.
    """
    if not np.iscomplexobj(radar_data):
        raise ValueError('Expected complex radar data for quality analysis')
    
    amplitudes = np.abs(radar_data)
    
    # Calculate quality metrics
    quality_metrics = {
        'peak_to_mean_ratio': amplitudes.max() / amplitudes.mean(),
        'dynamic_range_db': 20 * np.log10(amplitudes.max() / amplitudes.min()) if amplitudes.min() > 0 else float('inf'),
        'signal_to_noise_estimate': amplitudes.mean() / amplitudes.std(),
        'image_entropy': -np.sum(amplitudes * np.log(amplitudes + 1e-12)),
        'contrast': amplitudes.std() / amplitudes.mean(),
    }
    
    return quality_metrics


# Display comprehensive results only if focusing was successful
if 'focused_radar_data' in locals():
    post_focus_metadata = get_focusing_metadata(focuser)
    display_focusing_results(focused_radar_data, post_focus_metadata)
    
    # Analyze focusing quality
    print('\n🎯 Focusing Quality Analysis:')
    quality_metrics = analyze_focusing_quality(focused_radar_data)
    for metric, value in quality_metrics.items():
        print(f'  {metric}: {value:.4f}')
else:
    print('⏸️  Focusing not yet completed - run the focusing cell first.')

In [ ]:
def cleanup_memory(variables_to_keep: Optional[list] = None) -> None:
    """Clean up memory by deleting large variables and running garbage collection.
    
    Args:
        variables_to_keep (Optional[list]): List of variable names to preserve.
    """
    import gc
    
    variables_to_keep = variables_to_keep or ['focused_radar_data', 'focuser']
    
    # Get current globals
    current_globals = list(globals().keys())
    
    # Variables that are safe to delete (large data arrays)
    deletable_vars = ['echo', 'metadata', 'ephemeris', 'raw_data']
    
    deleted_count = 0
    for var_name in deletable_vars:
        if var_name in current_globals and var_name not in variables_to_keep:
            try:
                del globals()[var_name]
                deleted_count += 1
                print(f'  ✅ Deleted variable: {var_name}')
            except KeyError:
                pass
    
    # Run garbage collection
    collected = gc.collect()
    print(f'  🗑️  Garbage collector freed {collected} objects')
    print(f'  📝 Deleted {deleted_count} large variables')


# Final memory usage check
print('\n💾 Final Memory Usage:')
print_memory_usage()

print('\n🧹 Memory Cleanup (optional):')
print('Uncomment the line below to free memory from large intermediate variables:')
print('# cleanup_memory()')

# Uncomment to actually run cleanup
# cleanup_memory()